In [1]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")  

# Train the model
model.train(
    data="C:\\Users\\kayma\\Downloads\\Bar_Path_Tracking.v4i.yolov8\\data.yaml",
    epochs=20, 
    imgsz=416, 
    batch=4, 
    device="cpu"  
)


New https://pypi.org/project/ultralytics/8.3.161 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.91  Python-3.9.21 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13600KF)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Users\kayma\Downloads\Bar_Path_Tracking.v4i.yolov8\data.yaml, epochs=20, time=None, patience=100, batch=4, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mask

train: Scanning C:\Users\kayma\Downloads\Bar_Path_Tracking.v4i.yolov8\train\labels... 1606 images, 6 backgrounds, 0 corrupt: 100%|██████████| 1606/1606 [00:01<00:00, 1275.67it/s]


train: New cache created: C:\Users\kayma\Downloads\Bar_Path_Tracking.v4i.yolov8\train\labels.cache


val: Scanning C:\Users\kayma\Downloads\Bar_Path_Tracking.v4i.yolov8\valid\labels... 151 images, 0 backgrounds, 0 corrupt: 100%|██████████| 151/151 [00:00<00:00, 1392.66it/s]

val: New cache created: C:\Users\kayma\Downloads\Bar_Path_Tracking.v4i.yolov8\valid\labels.cache


...

                   all        151        169      0.993      0.994      0.995      0.762



20 epochs completed in 1.292 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 22.5MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics 8.3.91  Python-3.9.21 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-13600KF)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:03<00:00,  5.21it/s]


                   all        151        169      0.993      0.994      0.995      0.761
Speed: 0.1ms preprocess, 21.5ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001FFAA687AF0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

MoveNet


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO

# Loading the YOLOv8 model for barbell detection
yolo_model = YOLO("runs/detect/train3/weights/best.pt")

# Loading the MoveNet Thunder (TFLite) for Sqaut Pose Estimation
interpreter = tf.lite.Interpreter(model_path="movenet_thunder.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initalising Variables
barbell_path = [] # This will store the co-ordinates of the barbell path
angle_history = [] # This will store previous knee angles
rep_count = 0 # This will keep count of squat repetitions
squat_down = False # This will help detect squat phase transition
feedback_message = "" # This will give real time feedback to the user
bar_center_history = [] # This will store barbell positions for sway detection

# Thresholds
squat_down_angle = 90 # Knee angle threshold to mark squat descent
squat_up_angle = 160  # Knee angle threshold to mark squat ascent
max_sway = 50   # Maximum allowed barbell sway in pixels


# Angle Calculation
"""
    Calculates the angle between three points (a, b, c) in degrees.
    b is treated as the vertex point.
    This will be used to measure knee joint angles during squats.
"""

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180 else int(angle)

# Pose estimation + squat logic
"""
    Detects the squat pose using MoveNet, calculates knee angles,
    counts squat repetitions, and provides feedback on squat depth.
"""

def detect_pose(frame):
    global angle_history, rep_count, squat_down, feedback_message

    h, w, _ = frame.shape
    input_image = cv2.resize(frame, (256, 256))
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    input_image = np.expand_dims(input_image, axis=0).astype(np.uint8)

    interpreter.set_tensor(input_details[0]['index'], input_image)
    interpreter.invoke()
    keypoints = interpreter.get_tensor(output_details[0]['index'])[0][0]

    # Body skeleton
    skeleton = [
        (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),
        (5, 11), (6, 12), (11, 12), (11, 13), (13, 15),
        (12, 14), (14, 16)
    ]

    points = []
    for y, x, conf in keypoints:
        if conf > 0.3:
            cx, cy = int(x * w), int(y * h)
            points.append((cx, cy))
            cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
        else:
            points.append(None)

    for pair in skeleton:
        pt1, pt2 = points[pair[0]], points[pair[1]]
        if pt1 and pt2:
            cv2.line(frame, pt1, pt2, (0, 255, 255), 2)

    # Angle calculation
    try:
        l_hip, l_knee, l_ankle = points[11], points[13], points[15]
        r_hip, r_knee, r_ankle = points[12], points[14], points[16]

        angles = []
        if l_hip and l_knee and l_ankle:
            angles.append(calculate_angle(l_hip, l_knee, l_ankle))
        if r_hip and r_knee and r_ankle:
            angles.append(calculate_angle(r_hip, r_knee, r_ankle))

        if angles:
            avg_knee_angle = int(np.mean(angles))

            # Rep counter 
            if avg_knee_angle < squat_down_angle and not squat_down:
                squat_down = True
            if avg_knee_angle > squat_up_angle and squat_down:
                rep_count += 1
                squat_down = False

            # Feedback message
            if avg_knee_angle > squat_up_angle:
                feedback_message = "Too Upright"
            elif avg_knee_angle < 60:
                feedback_message = "Too Deep!"
            else:
                feedback_message = "Good Depth"

            # Display angle
            cv2.putText(frame, f"Knee Angle: {avg_knee_angle}°", (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    except:
        pass

    return frame

# Barbell tracking 
"""
    Detects the barbell position using YOLOv8, draws detection boxes,
    tracks bar path, and checks for horizontal sway.
"""

def track_barbell(frame, results):
    global barbell_path, feedback_message, bar_center_history

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        conf = box.conf[0]
        if conf > 0.4:
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            barbell_path.append((cx, cy))
            bar_center_history.append(cx)

            # Drawing bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.putText(frame, f"Barbell", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Drawing barbell path
    for i in range(1, len(barbell_path)):
        cv2.line(frame, barbell_path[i - 1], barbell_path[i], (0, 165, 255), 2)

    # Bar path stability check
    if len(bar_center_history) > 10:
        sway_range = max(bar_center_history[-10:]) - min(bar_center_history[-10:])
        if sway_range > max_sway:
            feedback_message = "Bar Sway Detected!"

    return frame

# Setup video
#cap = cv2.VideoCapture("barbell_video.mp4") - video of a person squating with a bar
#cap = cv2.VideoCapture("bodyweight_squat.mp4") - video of a person squating without a barbell
cap = cv2.VideoCapture("Back Squat - Side View.mp4")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_pose_squat_system.mp4", fourcc, 30,
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Pose detection and squat analysis
    pose_frame = detect_pose(frame.copy())

    # Barbell detection
    results = yolo_model(frame, imgsz=416)[0]
    pose_frame = track_barbell(pose_frame, results)

    # Displaying rep count and feedback
    cv2.putText(pose_frame, f"Reps: {rep_count}", (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    if feedback_message:
        cv2.putText(pose_frame, feedback_message, (30, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)

    # Showing and saving real-time video 
    cv2.imshow("Squat Analyzer", pose_frame)
    out.write(pose_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


c:\Users\kayma\anaconda3\envs\yolov8_env\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


0: 256x416 1 Bar-Path, 52.3ms
Speed: 2.6ms preprocess, 52.3ms inference, 1.1ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 1 Bar-Path, 42.3ms
Speed: 0.9ms preprocess, 42.3ms inference, 0.5ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 1 Bar-Path, 43.9ms
Speed: 0.7ms preprocess, 43.9ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 1 Bar-Path, 41.3ms
Speed: 0.8ms preprocess, 41.3ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 1 Bar-Path, 43.3ms
Speed: 0.8ms preprocess, 43.3ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 1 Bar-Path, 41.6ms
Speed: 0.7ms preprocess, 41.6ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 1 Bar-Path, 41.9ms
Speed: 0.7ms preprocess, 41.9ms inference, 0.4ms postprocess per image at shape (1, 3, 256, 416)

0: 256x416 1 Bar-Path, 41.8ms
Speed: 0.7ms preprocess, 41.8ms inference, 0.4ms postprocess per image at 

: 